# Import libraries

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from lib import *
from fun import *

# Import polygons

In [32]:
shapefilename = "data/polygons/TG00CAGRP.shp"
npoints_within_polygon = 5
blocks = fiona.open(shapefilename)
blocks_polygons = [shape(item["geometry"]) for item in blocks]

# Generate random points within polygons

In [41]:
d = AutoVivification()
for idx,randompolygon in enumerate(blocks_polygons[:]):
    clear_output(wait=True)
    within_points = Draw_Random_Points(randompolygon,npoints_within_polygon)
    d[idx] = {"points" : within_points}
    progression = round ( (idx + 1) / len(blocks_polygons) , 2)
    progression = 100 * progression
    progression = str(progression) + "%"
    print("Progression : ",progression)

Progression :  100.0%


# Save generated points

In [42]:
with open("data/polygon_within_points.json", "w") as f:
    json.dump(d,f)

In [ ]:
%%timeit
elevation = py3dep.elevation_bycoords([(-122, 44)], crs="epsg:4326")

In [4]:
elevation

[1434]

In [8]:
import requests
import urllib
import pandas as pd


In [9]:

# USGS Elevation Point Query Service
url = r'https://nationalmap.gov/epqs/pqs.php?'

# coordinates with known elevation 
lat = [48.633, 48.733, 45.1947, 45.1962]
lon = [-93.9667, -94.6167, -93.3257, -93.2755]

# create data frame
df = pd.DataFrame({
    'lat': lat,
    'lon': lon
})

In [10]:
def elevation_function(df, lat_column, lon_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):

        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }

        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        elevations.append(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])

    df['elev_meters'] = elevations
    return df


In [11]:
elevation_function(df, 'lat', 'lon')

,lat,lon,elev_meters
0,48.6330,-93.9667,341.14
1,48.7330,-94.6167,328.80
2,45.1947,-93.3257,262.68
3,45.1962,-93.2755,272.64
